## Тестовое задание

1. Определить процент доставляемости PUSH-рассылок в рекламных кампаниях за
3-4 кварталы 2023 года

In [ ]:
# По каждой кампании (кампания, дата старта кампании, % успешных доставок (delivered))

'''
    WITH raw AS ( -- для каждой коммуникации определённой кампании определяет флаг отправки
                  -- сообщения и флаг его доставки
                  SELECT  cmp.campaign_name,
                          cmp.started_ts,
                          cmm.communication_id,
                          MAX(CASE WHEN stt.status_name = 'sent' THEN 1 ELSE 0 END) AS sent_status_flag,
                          MAX(CASE WHEN stt.status_name = 'delivered' THEN 1 ELSE 0 END) AS delivered_status_flag

                  -- таблица с кампаниями
                  FROM Dim_campaigns AS cmp

                  -- таблица с каналами
                  -- доп. проверка на активность канала в период кампании
                  LEFT JOIN Dim_channels AS chn
                  ON cmp.channel_id = chn.ID
                  AND cmp.started_ts BETWEEN chn.row_actual_from_ts AND chn.row_actual_to
                  AND cmp.ended_ts BETWEEN chn.row_actual_from_ts AND chn.row_actual_to

                  -- таблица с шаблонами
                  -- доп. проверка на активность шаблона в период кампании
                  LEFT JOIN Dim_templates AS tmp
                  ON chn.ID = tmp.channel_id
                  AND cmp.started_ts BETWEEN tmp.row_actual_from_ts AND tmp.row_actual_to
                  AND cmp.ended_ts BETWEEN tmp.row_actual_from_ts AND tmp.row_actual_to

                  -- таблица с коммуникациями
                  -- объединение по кампании, шаблону и определению статуса коммуникации
                  -- в период проведения кампании
                  LEFT JOIN Campaign_communications AS cmm
                  ON cmp.id = cmm.campaign_id
                  AND tmp.id = cmm.template_id
                  AND cmm.status_ts BETWEEN cmp.started_ts AND cmp.ended_ts

                  -- таблица со статусами
                  -- доп. проверка на активность статуса в момент опеределения статуса коммуникации
                  LEFT JOIN Communication_statuses AS stt
                  ON stt.id = cmm.communication_status_id
                  AND cmm.status_ts BETWEEN stt.row_actual_from AND stt.row_actual_to

                  -- оставляет только push-рассылки, которые осуществлялись в 3-4 квартале 2023
                  WHERE chn.name = 'PUSH'
                  AND cmm.status_ts BETWEEN '2023-07-01' AND '2023-12-01'

                  GROUP BY 1, 2, 3

                  -- убирает ошибочные статусы, где есть флаг доставки, но нет флага отправления
                  HAVING MAX(CASE WHEN stt.status_name = 'delivered' THEN 1 ELSE 0 END)
                        <= MAX(CASE WHEN stt.status_name = 'sent' THEN 1 ELSE 0 END)
                  )

    SELECT  campaign_name,
            started_ts,

            -- опеределяет процент успешных доставок
            CASE
                WHEN SUM(delivered_status_flag) / SUM(sent_status_flag) * 100 IS NULL THEN 0
                ELSE SUM(delivered_status_flag) / SUM(sent_status_flag) * 100
            END AS delivered_percent

    FROM raw

    GROUP BY 1, 2
'''

In [ ]:
# В среднем по каналу за каждый месяц (месяц, % успешных доставок)

'''
    WITH raw AS ( -- для каждой коммуникации определённого канала и конкретного месяца
                  -- определяет флаг отправки сообщения и флаг его доставки
                  SELECT  date_trunc('month', cmm.status_ts) AS month,
                          chn.name,
                          cmm.communication_id,
                          MAX(CASE WHEN stt.status_name = 'sent' THEN 1 ELSE 0 END) AS sent_status_flag,
                          MAX(CASE WHEN stt.status_name = 'delivered' THEN 1 ELSE 0 END) AS delivered_status_flag

                  -- таблица с кампаниями
                  FROM Dim_campaigns AS cmp

                  -- таблица с каналами
                  -- доп. проверка на активность канала в период кампании
                  LEFT JOIN Dim_channels AS chn
                  ON cmp.channel_id = chn.ID
                  AND cmp.started_ts BETWEEN chn.row_actual_from_ts AND chn.row_actual_to
                  AND cmp.ended_ts BETWEEN chn.row_actual_from_ts AND chn.row_actual_to

                  -- таблица с шаблонами
                  -- доп. проверка на активность шаблона в период кампании
                  LEFT JOIN Dim_templates AS tmp
                  ON chn.ID = tmp.channel_id
                  AND cmp.started_ts BETWEEN tmp.row_actual_from_ts AND tmp.row_actual_to
                  AND cmp.ended_ts BETWEEN tmp.row_actual_from_ts AND tmp.row_actual_to

                  -- таблица с коммуникациями
                  -- объединение по кампании, шаблону и определению статуса коммуникации
                  -- в период проведения кампании
                  LEFT JOIN Campaign_communications AS cmm
                  ON cmp.id = cmm.campaign_id
                  AND tmp.id = cmm.template_id
                  AND cmm.status_ts BETWEEN cmp.started_ts AND cmp.ended_ts

                  -- таблица со статусами
                  -- доп. проверка на активность статуса в момент опеределения статуса коммуникации
                  LEFT JOIN Communication_statuses AS stt
                  ON stt.id = cmm.communication_status_id
                  AND cmm.status_ts BETWEEN stt.row_actual_from AND stt.row_actual_to

                  -- оставляет только push-рассылки, которые осуществлялись в 3-4 квартале 2023
                  WHERE chn.name = 'PUSH'
                  AND cmm.status_ts BETWEEN '2023-07-01' AND '2023-12-01'

                  GROUP BY 1, 2, 3

                  -- убирает ошибочные статусы, где есть флаг доставки, но нет флага отправления
                  HAVING MAX(CASE WHEN stt.status_name = 'delivered' THEN 1 ELSE 0 END)
                        <= MAX(CASE WHEN stt.status_name = 'sent' THEN 1 ELSE 0 END)
                  )

    SELECT  month,

            -- опеределяет процент успешных доставок
            CASE
                WHEN SUM(delivered_status_flag) / SUM(sent_status_flag) * 100 IS NULL THEN 0
                ELSE SUM(delivered_status_flag) / SUM(sent_status_flag) * 100
            END AS delivered_percent

    FROM raw

    GROUP BY 1
'''

2. Определить за каждую неделю с 1 мая 2023 года топ 10 событий, на которых заканчиваются пользовательские сессии.

In [ ]:
'''
    WITH raw AS ( -- считает количество совершённых событий для каждого типа события за каждую неделю
                SELECT  week,
                        year,
                        event_name,
                        SUM(RN) AS event_cnt

                -- определяет последнее событие для каждой сессии
                FROM ( SELECT *,
                              ROW_NUMBER() OVER(PARTITION BY user_id, session_id ORDER BY time_stamp DESC) AS RN,
                              EXTRACT(WEEK FROM time_stamp) AS week,
                              EXTRACT(YEAR FROM time_stamp) AS year

                        FROM Session_log AS l

                        LEFT JOIN dim_events AS e
                        ON l.event_id = e.id
                        AND l.time_stamp BETWEEN e.row_actual_from AND e.row_actual_to

                        WHERE l.time_stamp::date >= '2023-05-01' -- данные с 1 мая 2023 года
                        ) AS a

                WHERE RN = 1

                GROUP BY 1, 2, 3
                )

      -- для каждой недели выводит топ-10 совершённых последних событий
      SELECT  week,
              year,
              event_name,
              event_cnt,
              place

        FROM ( -- определяет место типа события за каждую неделю
              SELECT  *,
                      ROW_NUMBER() OVER(PARTITION BY week, year ORDER BY event_cnt DESC) AS place
              FROM raw
              ) AS b

        WHERE place <= 10

        ORDER BY 2, 1, 5
'''

3. Заказы на маркетплейсе

In [ ]:
# Выбрать топ-10 поставщиков по кол-ву успешных заказов (delivered) за последние полгода

'''
    SELECT  o.supplier_id,
            s.name AS supplier_name,
            COUNT(DISTINCT o.id) AS orders_cnt

    FROM orders AS o

    -- таблица с поставщиками
    -- доп. проверка, чтобы поставщик был активным на момент заказа
    LEFT JOIN suppliers AS s
    ON o.supplier_id = s.id
    AND o.order_created_datetime BETWEEN s.active_from AND s.active_to

    -- таблица с логами статусов заказа
    LEFT JOIN Order_status_log AS ord
    ON o.id = ord.order_id

    -- таблица со статусами
    -- доп. проверка, чтобы статус был активным на момент логирования
    LEFT JOIN Dim_statuses AS st
    ON ord.status_id = st.id
    AND ord.date_time BETWEEN st.row_actual_from AND st.row_actual_to

    -- оставляет только доставленные заказы за последние полгода
    WHERE st.status_name = 'Order_delivered'
    AND o.order_created_datetime::date >= now() - interval '6 month'

    GROUP BY 1, 2

    ORDER BY 3

    LIMIT 10
'''

In [ ]:
# Определить топ-10 самых быстрых поставщиков за последние 3 месяца с кол-вом успешных заказов от 50 шт в неделю.

'''
  WITH raw AS ( -- собирает все данные о заказах
                SELECT  o.id AS order_id,
                        o.order_created_datetime,
                        EXTRACT(WEEK FROM o.order_created_datetime) AS week,
                        EXTRACT(YEAR FROM o.order_created_datetime) AS year,
                        o.supplier_id,
                        s.name AS supplier_name,
                        st.status_name,
                        ord.date_time AS log_dt,
                        extract(julian from ord.date_time) - extract(julian from o.order_created_datetime) AS dt_diff

                FROM suppliers AS s

                -- таблица с заказами
                -- доп. проверка, чтобы поставщик был активным на момент заказа
                LEFT JOIN orders AS o
                ON o.supplier_id = s.id
                AND o.order_created_datetime BETWEEN s.active_from AND s.active_to

                -- таблица с логами статусов заказа
                LEFT JOIN Order_status_log AS ord
                ON o.id = ord.order_id

                -- таблица со статусами
                -- доп. проверка, чтобы статус был активным на момент логирования
                LEFT JOIN Dim_statuses AS st
                ON ord.status_id = st.id
                AND ord.date_time BETWEEN st.row_actual_from AND st.row_actual_to

                -- оставляет заказы за 3 последних месяца
                WHERE status_name = 'Order_delivered'
                AND o.order_created_datetime::date >= now() - interval '3 month'
                ),

    supl AS ( -- определяет поставщиков, еженедельно имеющих как минимум 50 заказов
             SELECT supplier_id

             FROM (SELECT year,
                          week,
                          supplier_id,
                          COUNT(order_id) AS orders_cnt

                  FROM raw

                  GROUP BY 1, 2, 3
                  HAVING COUNT(order_id) >= 50
                  ) AS sp

              GROUP BY 1
              -- количество недель с успешными заказами не меньше,
              -- чем общее количество недель в сырых данных
              HAVING COUNT(week) >= (SELECT COUNT(DISTINCT CONCAT(year || week)) FROM raw)
              )

      -- выводит топ-10 поставщиков с наибольшей средней скоростью даставки
      SELECT  supplier_id,
              supplier_name,
              AVG(dt_diff) AS avg_dt_diff

      FROM raw

      -- только те поставщики, у которых как минимум 50 заказов еженедельно
      WHERE supplier_id IN (SELECT supplier_id FROM spl)

      GROUP BY 1, 2

      ORDER BY 3

      LIMIT 10
'''

In [ ]:
# Определить топ 10-поставщиков (suppliers) c наибольшим кол-во отмени возвратов за последние полгода.

'''
    SELECT  o.supplier_id,
            s.name AS supplier_name,
            COUNT(DISTINCT o.id) AS orders_cnt

    FROM orders AS o

    -- таблица с поставщиками
    -- доп. проверка, чтобы поставщик был активным на момент заказа
    LEFT JOIN suppliers AS s
    ON o.supplier_id = s.id
    AND o.order_created_datetime BETWEEN s.active_from AND s.active_to

    -- таблица с логами статусов заказа
    LEFT JOIN Order_status_log AS ord
    ON o.id = ord.order_id

    -- таблица со статусами
    -- доп. проверка, чтобы статус был активным на момент логирования
    LEFT JOIN Dim_statuses AS st
    ON ord.status_id = st.id
    AND ord.date_time BETWEEN st.row_actual_from AND st.row_actual_to

    -- оставляет только отменённые заказы или возвраты за последние полгода
    WHERE st.status_name IN ( 'Order_cancelled_by_customer',
                              'Order_cancelled_by_supplier',
                              'Refund',
                              'Money_returned' )
    AND o.order_created_datetime::date - interval '6 month'

    GROUP BY 1, 2

    ORDER BY 3

    LIMIT 10
'''

In [ ]:
# Посчитать процент возвратов по каждому поставщику за 2024 год помесячно.
'''
                -- определяет общее количество заказов и возвратов на поставщика в месяц
                SELECT  date_trunc('Year', o.order_created_datetime) AS month,
                        o.supplier_id,
                        s.name AS supplier_name,
                        MAX(CASE WHEN st.status_name = 'Refund' THEN 1 ELSE 0 END) / COUNT(DISTINCT o.id) * 100 AS refund_percent

                FROM orders AS o

                -- таблица с поставщиками
                -- доп. проверка, чтобы поставщик был активным на момент заказа
                LEFT JOIN suppliers AS s
                ON o.supplier_id = s.id
                AND o.order_created_datetime BETWEEN s.active_from AND s.active_to

                -- таблица с логами статусов заказа
                LEFT JOIN Order_status_log AS ord
                ON o.id = ord.order_id

                -- таблица со статусами
                -- доп. проверка, чтобы статус был активным на момент логирования
                LEFT JOIN Dim_statuses AS st
                ON ord.status_id = st.id
                AND ord.date_time BETWEEN st.row_actual_from AND st.row_actual_to

                -- оставляет заказы только за 2024 год
                WHERE EXTRACT(YEAR FROM o.order_created_datetime) = '2024'

                GROUP BY 1, 2, 3
    '''